# 🗺️ Kayak Travel Dashboard: Best Destinations in France 
**Executive Summary & Project Deliverables**

This notebook serves as the final presentation layer for the Data Engineering pipeline. It demonstrates the successful completion of all three project deliverables:
1. **Cloud Storage (Data Lake):** Retrieving the enriched dataset directly from an AWS S3 bucket.
2. **Relational Database (Data Warehouse):** Querying the final cleaned data from an AWS RDS PostgreSQL instance.
3. **Interactive Dashboards:** Visualizing the best destinations and hotels using Plotly.

In [10]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import plotly.express as px
import textwrap

# Load credentials from .env file
load_dotenv(dotenv_path="../.env")

print("✅ Libraries loaded and environment variables secured!")

✅ Libraries loaded and environment variables secured!


## ☁️ Deliverable 1: AWS S3 Data Lake
First, we will prove that our final enriched dataset (`kayak_master.csv`) is safely stored in the cloud. We will use Pandas to read the file directly from the S3 bucket using our AWS credentials.

In [14]:
# Fetch AWS credentials
aws_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret = os.getenv("AWS_SECRET_ACCESS_KEY")
bucket_name = "dsfs-1-enavarr-project-kayak" # ⚠️ If your are running the notebook Update this with your own bucket name!

s3_file_path = f"s3://{bucket_name}/kayak_master.csv"

print(f"📥 Fetching data directly from {s3_file_path}...")

# Read directly from S3
df_s3 = pd.read_csv(
    s3_file_path,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    }
)

# Display the first 3 rows to prove it worked
display(df_s3.head(3))

📥 Fetching data directly from s3://dsfs-1-enavarr-project-kayak/kayak_master.csv...


,city_id,city,hotel_name,url,score,description,hotel_lat,hotel_lon,weather_score,climate_index,avg_temp,total_rain_mm,latitude,longitude
0,28,Collioure,location Coma Chéric et parking,https://www.booking.com/hotel/fr/location-coma...,9.79,Beachfront Location: The apartment offers dire...,42.522981,3.087304,72.1744,36.3272,17.404,0.0,42.52505,3.083155
1,28,Collioure,Les mimosas,https://www.booking.com/hotel/fr/les-mimosas-c...,9.69,Elegant Accommodations: Les mimosas in Colliou...,42.524325,3.077175,72.1744,36.3272,17.404,0.0,42.52505,3.083155
2,28,Collioure,"Apartment Olive & Kin, Collioure",https://www.booking.com/hotel/fr/residence-por...,9.59,Essential Facilities: Apartment Olive & Kin in...,42.522267,3.088695,72.1744,36.3272,17.404,0.0,42.52505,3.083155


## 🗄️ Deliverable 2: AWS RDS PostgreSQL Database
Next, we will connect to our live data warehouse. The ETL pipeline has already structured and pushed our data here. We will query the database to retrieve the master table.

In [15]:
# Fetch Database credentials matching your .env file
DB_USER = os.getenv("AWS_RDS_USER")
DB_PASSWORD = os.getenv("AWS_RDS_PASSWORD")
DB_HOST = os.getenv("AWS_RDS_HOST")
DB_PORT = os.getenv("AWS_RDS_PORT", "5432")
DB_NAME = os.getenv("AWS_RDS_DB_NAME")

# Create connection engine
# Note: Adding postgresql+psycopg2:// to match your working script
conn_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_string)

print("🔌 Connecting to AWS RDS...")

# Query the database
# Make sure "destinations" matches the table name you used in etl_sql.py
query = "SELECT * FROM destinations;" 
df_sql = pd.read_sql(query, engine)

print(f"✅ Successfully retrieved {len(df_sql)} rows from the database!")
display(df_sql.head(3))

🔌 Connecting to AWS RDS...
✅ Successfully retrieved 700 rows from the database!


,city_id,city,hotel_name,url,score,description,hotel_lat,hotel_lon,weather_score,climate_index,avg_temp,total_rain_mm,latitude,longitude
0,28,Collioure,location Coma Chéric et parking,https://www.booking.com/hotel/fr/location-coma...,9.79,Beachfront Location: The apartment offers dire...,42.522981,3.087304,72.1744,36.3272,17.404,0.0,42.52505,3.083155
1,28,Collioure,Les mimosas,https://www.booking.com/hotel/fr/les-mimosas-c...,9.69,Elegant Accommodations: Les mimosas in Colliou...,42.524325,3.077175,72.1744,36.3272,17.404,0.0,42.52505,3.083155
2,28,Collioure,"Apartment Olive & Kin, Collioure",https://www.booking.com/hotel/fr/residence-por...,9.59,Essential Facilities: Apartment Olive & Kin in...,42.522267,3.088695,72.1744,36.3272,17.404,0.0,42.52505,3.083155


## 🗺️ Deliverable 3: Interactive Dashboards
Finally, we visualize the data retrieved from our SQL database to answer the core business question: **Where should we travel, and where should we sleep?**

In [16]:
# --- PREP DATA FOR MAP 1 ---
# Group by city to get the destination stats
city_stats = df_sql.groupby('city').agg({
    'latitude': 'first',
    'longitude': 'first',
    'avg_temp': 'first',
    'total_rain_mm': 'first',
    'climate_index': 'first', 
    'weather_score': 'first'  
}).reset_index()

city_stats = city_stats.sort_values(by='weather_score', ascending=False)
city_stats['plot_size'] = city_stats['weather_score'].apply(lambda x: max(x + 5, 2))

# 🌟 Add the Trophies for the Top 5
city_stats['highlight_label'] = "" 
top_5_indices = city_stats.head(5).index
for idx in top_5_indices:
    city_stats.loc[idx, 'highlight_label'] = f"🏆 {city_stats.loc[idx, 'city']}"

# --- DRAW MAP 1 ---
fig1 = px.scatter_map(
    city_stats, 
    lat="latitude", 
    lon="longitude",
    color="climate_index",      
    size="plot_size",           
    text="highlight_label",     
    hover_name="city",
    hover_data={
        "climate_index": False, 
        "plot_size": False,
        "highlight_label": False, 
        "weather_score": ':.1f', 
        "avg_temp": ':.1f', 
        "total_rain_mm": ':.1f'
    },
    zoom=5,
    map_style="carto-positron", 
    color_continuous_scale="Turbo", 
    range_color=[0, 100],           
    title="Destinations Ranked by Weather (Turbo Scale: Blue=Cold, Green=Perfect, Red=Hot)",
    height=600 # Make it nice and big for the presentation
)

fig1.update_traces(
    textposition='top center',
    textfont=dict(size=14, color='black', weight='bold')
)

# 🏆 Build the Leaderboard Box
leaderboard_text = "<b>🏆 Top 5 Destinations</b><br>"
for i, (_, row) in enumerate(city_stats.head(5).iterrows(), 1):
    leaderboard_text += f"{i}. {row['city']} (Score: {row['weather_score']:.1f})<br>"
    
fig1.add_annotation(
    text=leaderboard_text, align='left', showarrow=False,
    xref='paper', yref='paper', x=0.98, y=0.02, xanchor='right', yanchor='bottom',
    bgcolor='rgba(255, 255, 255, 0.85)', bordercolor='black', borderwidth=1, borderpad=10
)

fig1.show()

In [19]:
import pandas as pd
import textwrap
import plotly.express as px

# --- PREP DATA FOR MAP 2 ---
# Get the names of the Top 5 cities
top_5_city_names = city_stats.head(5)['city'].tolist()

# Filter the master SQL dataframe for ONLY those 5 cities
top_hotels = df_sql[df_sql['city'].isin(top_5_city_names)].copy()

# 🛠️ THE FIX: Convert 'score' from text to numbers
top_hotels['score'] = pd.to_numeric(top_hotels['score'], errors='coerce')

# Sort to find the absolute best hotels
top_hotels = top_hotels.sort_values(by='score', ascending=False)
top_hotels = top_hotels.dropna(subset=['hotel_lat', 'hotel_lon', 'score'])

# ✂️ THE FIX: Truncate to 150 chars, THEN wrap every 50 chars onto a new line
def format_description(text):
    if not isinstance(text, str):
        return str(text)
    # 1. Truncate
    truncated = text[:150] + "..." if len(text) > 150 else text
    # 2. Wrap at 50 characters (creates \n newlines)
    wrapped = textwrap.fill(truncated, width=50)
    # 3. Replace standard \n with HTML <br> for Plotly
    return wrapped.replace('\n', '<br>')

# Apply the wrapping function to create a new column for the hover text
top_hotels['short_desc'] = top_hotels['description'].apply(format_description)

# --- DRAW MAP 2 ---
fig2 = px.scatter_map(
    top_hotels,
    lat="hotel_lat",
    lon="hotel_lon",
    color="score",
    size="score", 
    hover_name="hotel_name",
    hover_data={
        "hotel_lat": False, 
        "hotel_lon": False,
        "city": True,
        "score": True,
        "short_desc": True,    # Show our wrapped text
        "description": False   # Hide the giant original text
    },
    labels={
        "short_desc": "Description" # Make it look professional in the tooltip
    },
    zoom=5,
    map_style="carto-positron",
    color_continuous_scale="Viridis",
    title="Top Hotels in the Top 5 Destinations",
    height=600
)

# 

# 🏆 Build the Context Box
context_text = "<b>📍 Showing Hotels For:</b><br><b>Top 5 Destinations</b><br>"
for i, city_name in enumerate(top_5_city_names, 1):
    # Fetch the score for this city from our city_stats dataframe
    city_score = city_stats[city_stats['city'] == city_name]['weather_score'].values[0]
    context_text += f"{i}. {city_name} (Score: {city_score:.1f})<br>"
    
fig2.add_annotation(
    text=context_text, align='left', showarrow=False,
    xref='paper', yref='paper', x=0.98, y=0.02, xanchor='right', yanchor='bottom',
    bgcolor='rgba(255, 255, 255, 0.85)', bordercolor='black', borderwidth=1, borderpad=10
)

fig2.show()